# Setup software libraries

Authenticate and import as necessary.

In [0]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

Use `%tensorflow_version 1.x` to keep using v1, otherwise the default tensorflow version is 2

In [0]:
# Tensorflow setup.
import tensorflow as tf
%tensorflow_version 1.x
tf.enable_eager_execution()
print(tf.__version__)

# Variables

Declare the variables that will be in use throughout the notebook.

## Specify the Cloud Storage Bucket

In [0]:
# INSERT YOUR BUCKET HERE:
BUCKET = 'sugarcane-unet-model'

## Define the loss function and the one of the metrics

In [0]:
from tensorflow.python.keras import backend

def dice_coeff(y_true, y_pred, smooth=1):
    y_true_f = backend.flatten(y_true)
    y_pred_f = backend.flatten(y_pred)
    intersection = backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (backend.sum(y_true_f) + backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

## Set other global variables

In [0]:
from tensorflow.python.keras import losses
from tensorflow.python.keras import metrics

# Specify names of output locations in Cloud Storage.
JOB_DIR = 'gs://' + BUCKET + '/modified_trainer_128_25_Adam_16'
MODEL_DIR = JOB_DIR + '/model'
LOGS_DIR = JOB_DIR + '/logs'
DATA_DIR = 'gs://' + BUCKET + '/data_128_NormalizedData'

# Pre-computed training and eval data.
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# Specify inputs to the model and the response variable.
opticalBands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
thermalBands = ['thermal'] # currently has some issues so not including
thermalBands = []
otherBands = [
  'p20_green', 'p20_nir', 'p20_blue', 'p20_red', 'p20_swir1', 'p20_swir2',
  'p80_green', 'p80_nir', 'p80_blue', 'p80_red', 'p80_swir1', 'p80_swir2',
]
indices = [
  'NDVI', 'EVI', 'SAVI', 'IBI', 'GRatio',
  'brightness', 'greenness', 'wetness', 'fourth', 'fifth', 'sixth',
]
BANDS = opticalBands + thermalBands + otherBands + indices
RESPONSE = 'sugarcane'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 128
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 16000
EVAL_SIZE = 8000

# Specify model training parameters.
BATCH_SIZE = 16
EPOCHS = 25
BUFFER_SIZE = 2000
OPTIMIZER = 'Adam'
LOSS = dice_loss
METRICS = [
    metrics.get('RootMeanSquaredError'),
    metrics.get('MeanAbsoluteError'),
    metrics.get('Accuracy'),
    dice_coeff,
]

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [0]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  glob = tf.gfile.Glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

Use the helpers to read in the training dataset.

In [0]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	glob = DATA_DIR + '/' + TRAINING_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()

# Evaluation data

Now do the same thing to get an evaluation dataset.  Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [0]:
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns: 
    A tf.data.Dataset of evaluation data.
  """
	glob = DATA_DIR + '/' + EVAL_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()

# Data Augmentation

Performing the data augmentation to increase the number of training datasets 

In [0]:
def upDownFlipInputs(inputs, labels):
    return tf.image.flip_up_down(inputs), tf.image.flip_up_down(labels)

def leftRightFlipInputs(inputs, labels):
    return tf.image.flip_left_right(inputs), tf.image.flip_left_right(labels)

def transposeInputs(inputs, labels):
    upDown = tf.image.flip_up_down(inputs)
    transpose = tf.image.flip_left_right(upDown)
    upDownLabel = tf.image.flip_up_down(labels)
    transposeLabel = tf.image.flip_left_right(upDownLabel)
    return transpose, transposeLabel

def rotateInputs(inputs, labels):        
    return tf.image.rot90(inputs), tf.image.rot90(labels)

In [0]:
training = training.concatenate(training.map(upDownFlipInputs))\
                   .concatenate(training.map(leftRightFlipInputs))\
                   .concatenate(training.map(transposeInputs))\
                   .concatenate(training.map(rotateInputs))

training = training.shuffle(buffer_size = BATCH_SIZE * 10)

# Model

Here we use the Keras implementation of the U-Net model.  The U-Net model takes any pixel patches as input and outputs per-pixel class probability, label or a continuous output. Add in the regularizer and/or dropout as required. Currently they are commented out.

In [0]:
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers
from tensorflow.python.keras import regularizers

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	#encoder = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	#encoder = layers.Dropout(0.2)(encoder)

	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	#encoder = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.01))(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	#encoder = layers.Dropout(0.2)(encoder)

    return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	#decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
    #decoder = layers.Dropout(0.2)(decoder)
    
    decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	#decoder = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.01))(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
    #decoder = layers.Dropout(0.2)(decoder)

    decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	#decoder = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.01))(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
    #decoder = layers.Dropout(0.2)(decoder)

    return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)])
	encoder0_pool, encoder0 = encoder_block(inputs, 16)
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 32)
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 64)
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 128)
	center = conv_block(encoder3_pool, 256) # center
	decoder3 = decoder_block(center, encoder3, 128)
	decoder2 = decoder_block(decoder3, encoder2, 64)
	decoder1 = decoder_block(decoder2, encoder1, 32)
	decoder0 = decoder_block(decoder1, encoder0, 16)
	outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER), 
		loss=LOSS,
		metrics=METRICS
	)

	return model

In [0]:
model = get_model()
model.summary()

# Training the model

You train a Keras model by calling `.fit()` on it.  Here we're going to train for EPOCHS epochs, which is suitable for demonstration purposes.
Some callbacks have been commented out. Use them as required.

In [0]:
m = get_model()
m.fit(
    x=training,
    epochs=EPOCHS, 
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
    validation_data=evaluation,
    validation_steps=EVAL_SIZE,
    callbacks=[
        tf.keras.callbacks.TensorBoard(LOGS_DIR),
        #tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4),
        #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1)
    ]
)

Save the model for future use.

In [ ]:
tf.contrib.saved_model.save_keras_model(m, MODEL_DIR)